[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1MjzUke5SDxffqmpruiUzfdBLR6gKbCBM)

<br>

< [7. Rozwiązanie bazowe](PD_rozw_bazowe.ipynb) | [9. Trening zmodyfikowanych modeli](PD_tren_zmod_modeli.ipynb) >


## [8. Modyfikacje]()

### 8.1 Inicjalizacja modelu EfficienNet

Zamnim siec została użyta do reidentyfikacji zainicjalizowano ją oraz przetestowano używając kodu poniżej.

EfficientNet b0

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
model_efficient = EfficientNet.from_pretrained('efficientnet-b0')
path_to_models = './models/efficientnet-b0.pth'
torch.save(model_efficient.state_dict(), path_to_models)

EfficientNet b7

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
model_efficient = EfficientNet.from_pretrained('efficientnet-b7')
path_to_models = './models/efficientnet-b7.pth'
torch.save(model_efficient.state_dict(), path_to_models)

### 8.2 Użycie jako sieci bazowej sieci EfficientNet

Główną modyfikacją jaka została wytestowana w tej pracy było użycie jako sieci bazowej do ekstrakcji cech, sieci EfficientNet. Ten model nie został zaimplementowany i przetestowany w bazowej wersji frameworku. Wiecej na temat powodów wyboru tej sieci przedstawiono w punkcie 5.2

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/EfficientNetBackBone.png?raw=true" alt="drawing" width="800"/>
<br><br>

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/TrainEfficientNet.png?raw=true" alt="drawing" width="800"/>
<br><br>

### 8.3 Zmiejszenie rozmiaru embeddingu

W celu zmniejszenia rozmaru generowanego embeddingu zdecydowano o zmniejszeniu rozmaru ostatniej warstwy sieci z 2048 parametrów do 512. Ma to na celu zmniejszenie rozmiaru ostatecznie zapisywanych danych oraz przyspieszenie procesu przeszukiwania wygenerowanych wektorów.

### 8.4 Zminiejszenie precyzji zapisywanych parametrów do float16

Zmniejszenie precyzji z jaką zpisywane są parametry w sieci jest istotne dla optymalizacji procesu obliczeń na urządzeniach producenta kart graficznych N-vidia oraz dla samego rozmiaru zapisanego modelu. Poniżej fragmet zmian w kodzie zapewniający transformację precyzji do float16

<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/ZmniejszeniePrecyzji.png?raw=true" alt="drawing" width="800"/>
<br><br>

### 8.5 Wprowadzenie dodatkowej augumentacji danych

#### Komórki Pomocnicze

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import random
import torch
class AddingNoise(object):

    def __init__(self,probability=0.3, level=0.01):
        self.level = level
        self.probability = probability

    def __call__(self, img_tensor):
      if random.uniform(0, 1) >= self.probability:
            return img_tensor
      delta = torch.rand(img_tensor.shape)
      delta.data.clamp_(-self.level, self.level)

      img_tensor = img_tensor + delta
      return img_tensor

In [ ]:
class Market1501Dataset:
  def __init__(self, path, img_list, transform):
    assert os.path.exists(path)
    self.path = path
    self.img_list = img_list
    self.transform = transform
    
    self.image_paths =[]    # Ścieżki do kolejnych obrazów
    self.class_names = []   # Nazwa klasy każdego obrazu
    
    for img_name in self.img_list:
      img_path = os.path.join(path, img_name)
      assert os.path.exists(img_path)
      self.image_paths.append(img_path)
      pos = img_name.find('_')      # Znajdź pierwsze wystąpienie '_'
      class_name = img_name[:pos]     # 4 pierwsze znaki w nazwie pliku są identyfikatorem klasy
      self.class_names.append(int(class_name))

    self.n_classes = len(set(self.class_names)) # Liczba klas (liczba różnych osób)

  def __len__(self):
    return len(self.image_paths)
    
  def __getitem__(self, ndx):
    image_path = self.image_paths[ndx]
    im = Image.open(image_path)
    _, file_name = os.path.split(image_path)
    image = np.asarray(im)
    if self.transform is not None:
      image = self.transform(image)

    return image, self.class_names[ndx]

  def print_info(self):
      print('Zbiór danych zawiera {} obrazów z {} klas'.format(len(self), self.n_classes))

In [ ]:
import os
import shutil
from torchvision.transforms import Compose, ToTensor, Normalize, ToPILImage
path_tt = './gdrive/My Drive/COCO_Darknet/yolov5s_miki_coco_persons_trained/detections_with_json/cropped'
os.path.exists(path_tt)

files = os.listdir(path_tt)
train_transform_tt = Compose([ToTensor()])
tt = Market1501Dataset(path_tt, files, train_transform_tt)
print(files.__len__())
print(tt)


1738


In [ ]:
import numpy as np
img = next(iter(tt))

In [ ]:
def tensor_to_image_tt(x):
  # Konwersja obrazu danego jako znormalizowany tensor do PIL Image
  transform = Compose([ToPILImage()])
  image = transform(x.cpu())
  return image

#### Komórki Pomocnicze

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import random
import torch
class AddingNoise(object):

    def __init__(self,probability=0.3, level=0.01):
        self.level = level
        self.probability = probability

    def __call__(self, img_tensor):
      if random.uniform(0, 1) >= self.probability:
            return img_tensor
      delta = torch.rand(img_tensor.shape)
      delta.data.clamp_(-self.level, self.level)

      img_tensor = img_tensor + delta
      return img_tensor

In [ ]:
class Market1501Dataset:
  def __init__(self, path, img_list, transform):
    assert os.path.exists(path)
    self.path = path
    self.img_list = img_list
    self.transform = transform
    
    self.image_paths =[]    # Ścieżki do kolejnych obrazów
    self.class_names = []   # Nazwa klasy każdego obrazu
    
    for img_name in self.img_list:
      img_path = os.path.join(path, img_name)
      assert os.path.exists(img_path)
      self.image_paths.append(img_path)
      pos = img_name.find('_')      # Znajdź pierwsze wystąpienie '_'
      class_name = img_name[:pos]     # 4 pierwsze znaki w nazwie pliku są identyfikatorem klasy
      self.class_names.append(int(class_name))

    self.n_classes = len(set(self.class_names)) # Liczba klas (liczba różnych osób)

  def __len__(self):
    return len(self.image_paths)
    
  def __getitem__(self, ndx):
    image_path = self.image_paths[ndx]
    im = Image.open(image_path)
    _, file_name = os.path.split(image_path)
    image = np.asarray(im)
    if self.transform is not None:
      image = self.transform(image)

    return image, self.class_names[ndx]

  def print_info(self):
      print('Zbiór danych zawiera {} obrazów z {} klas'.format(len(self), self.n_classes))

In [ ]:
import os
import shutil
from torchvision.transforms import Compose, ToTensor, Normalize, ToPILImage
path_tt = './gdrive/My Drive/COCO_Darknet/yolov5s_miki_coco_persons_trained/detections_with_json/cropped'
os.path.exists(path_tt)

files = os.listdir(path_tt)
train_transform_tt = Compose([ToTensor()])
tt = Market1501Dataset(path_tt, files, train_transform_tt)
print(files.__len__())
print(tt)


1738


In [ ]:
import numpy as np
img = next(iter(tt))

In [ ]:
def tensor_to_image_tt(x):
  # Konwersja obrazu danego jako znormalizowany tensor do PIL Image
  transform = Compose([ToPILImage()])
  image = transform(x.cpu())
  return image

Próbą zwiekszenia prezycji wyznaczanych embeddingów 
<br><br>
<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/baselineChanges.png?raw=true" alt="drawing" width="800"/>
<br><br>

### 8.6 Wprowadzenie funkcji Swish jako funkcji aktywacji

Funkcja Swish zdefiniowana jako: 
<br><br>
$\text{Swish}(x) = x \cdot (1 - {e}^{-x})^{-1}$
<br><br>
<img src="https://github.com/tomektarabasz/Praca_Dyplomowa_Tomasz_Tarabasz/blob/master/img/Swish_plot.jpg?raw=true" alt="drawing" width="400"/>
<br><br>

Zgodnie z [[3] Why Swish could perform better than ReLu](https://www.machinecurve.com/index.php/2019/05/30/why-swish-could-perform-better-than-relu/?fbclid=IwAR1muCJQpjU4M4dHcCx2wj3PAtYtL6rpEi5-aU9LSjrkrBONQXTqScR2AtU) taka postać funkcji aktywacji może poprawić wyniki poprzez:
 - Dla dużych wartości ujemnych, tak jak funkcja ReLu zeruje takie aktywacje.
 - Dla wartości dodatnich aktywacji funkcja Swish zachowuje się podobnie do ReLu i nie ograniczna tych wartości "od góry"
 - W okolicjach zera funkcja jest różniczkowalna i nie zawiera nielinowości
 - Małe wartości ujemne nie są zerowane co może pomóc w doszkalaniu modelu w procesie szukania subtelnych cech głębokich.

## Nawigacja

<br>

< [7. Rozwiązanie bazowe](PD_rozw_bazowe.ipynb) | [9. Trening zmodyfikowanych modeli](PD_tren_zmod_modeli.ipynb) >
